# Summary
This notebook removes comments containing words irrelevant to Title IX from a databse. The database was web scraped from the regulations.gov Title IX public request for comments.

The cleaned data is then saved for use in later notebooks.

In [83]:
import json
import pandas
from pandas.io.json import json_normalize
import urllib.request
import textract
import tempfile

In [54]:
# Load in the database file
json_data = json.load(open('./data/db3.json'))
data = json_normalize(json_data)

# Drop unneeded labels in the dataframe
data.drop(labels=['doc._id', 'doc._rev', 'id', 'key', 'value.rev'], inplace=True, axis='columns')

Downloading any attachments the comments might have had and copying the text into the comment body.

In [96]:
%%time
has_attachment = data.dropna(subset=['doc.attachment_download -href'])

user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:58.0) Gecko/20100101 Firefox/58.0'
headers = {'User-Agent': user_agent}

def get_attachment(url):
    try:
        print(url)
        request = urllib.request.Request(url, headers=headers)
        response = urllib.request.urlopen(request).read()
        temp = tempfile.NamedTemporaryFile(suffix='.pdf')
        temp.write(response)
        text = textract.process(temp.name)
        return text
    except:
        return ''

has_attachment['doc.attachment_download'] = has_attachment['doc.attachment_download -href'].map(get_attachment)

CPU times: user 1min 34s, sys: 1min 6s, total: 2min 40s
Wall time: 2h 55min 13s


/mnt/c/Users/nish/projects/titleix/env/lib/python3.5/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [97]:
with pandas.option_context('display.max_colwidth', 5000):
    display(has_attachment)

doc.attachment_download   \
2406                             
2425                             
2427                             
2428                             
5488                             
5489                             
5490                             
5492                             
6532                             
6533                             
8178                             
8212                             
8228                             
8236                             
8238                             
8240                             
8248                             
8250                             
8276                             
8286                             
8289                             
8290                             
8296                             
8321                             
8322                             
8331                             
8332                             
8339                             
8350                             
8372                             
...                        ...   
15303                            
15305                            
15307                            
15308                            
15309                            
15330                            
15331                            
15332                            
15338                            
15396                            
15422                            
15438                            
15443                            
15457                            
15481                            
15484                            
15495                            
15550                            
15561                            
15573                            
15578                            
15585                            
15591                            
15595                            
15609                            
16150                            
16211                            
16213                            
16324                            
16349                            

                                                                                           doc.attachment_download -href  \
2406      https://www.regulations.gov/contentStreamer?documentId=ED-2017-OS-0074-5955&attachmentNumber=1&contentType=pdf   
2425      https://www.regulations.gov/contentStreamer?documentId=ED-2017-OS-0074-5936&attachmentNumber=1&contentType=pdf   
2427      https://www.regulations.gov/contentStreamer?documentId=ED-2017-OS-0074-5934&attachmentNumber=1&contentType=pdf   
2428      https://www.regulations.gov/contentStreamer?documentId=ED-2017-OS-0074-5933&attachmentNumber=1&contentType=pdf   
5488      https://www.regulations.gov/contentStreamer?documentId=ED-2017-OS-0074-2875&attachmentNumber=1&contentType=pdf   
5489      https://www.regulations.gov/contentStreamer?documentId=ED-2017-OS-0074-2875&attachmentNumber=2&contentType=pdf   
5490      https://www.regulations.gov/contentStreamer?documentId=ED-2017-OS-0074-2875&attachmentNumber=4&contentType=pdf   
5492      https://www.regulations.gov/contentStreamer?documentId=ED-2017-OS-0074-2873&attachmentNumber=1&contentType=pdf   
6532      https://www.regulations.gov/contentStreamer?documentId=ED-2017-OS-0074-1833&attachmentNumber=1&contentType=pdf   
6533      https://www.regulations.gov/contentStreamer?documentId=ED-2017-OS-0074-1832&attachmentNumber=1&contentType=pdf   
8178     https://www.regulations.gov/contentStreamer?documentId=ED-2017-OS-0074-16365&attachmentNumber=1&contentType=pdf   
8212     https://www.regulations.gov/contentStreamer?documentId=ED-2017-OS-0074-16332&attachmentNumber=1&contentType=pdf   
8228   https://www.regulations.gov/contentStreamer?documentId=ED-2017-OS-0074-16316&attachmentNumber=1&contentType=msw12   
8236     https://www.regulations.gov/contentStreamer?documentId=ED-2017-OS-0074-16309&attachmentNumber=1&contentType=pdf   
8238     https://www.regulations.gov/

Checking the most common duplicate comments in the dataset.

In [4]:
duplicate_data = data.groupby(['doc.comment_body']).size().reset_index(name='duplicate_count')
duplicate_data = duplicate_data.sort_values('duplicate_count', ascending=False).reset_index(drop=True)

with pandas.option_context('display.max_colwidth', 200):
    display(duplicate_data[:5])

,doc.comment_body,duplicate_count
0,,1011
1,See attached file(s),294
2,"I'm requesting that the Dear Colleague Letter: ""Dyslexia Guidance"" dated October 23, 2015 (found at https://www2.ed.gov/policy/speced/guid/idea/memosdcltrs/guidance-on-dyslexia-10-2015.pdf) be ret...",44
3,"Dear Assistant General Counsel Hilary Malawer,\n\nCurrent federal regulations and guidance help all studentsregardless of sex, race, color, sexual orientation, gender identity, national origin, or...",42
4,"Dear Assistant General Counsel Hilary Malawer,\n\nCurrent federal regulations and guidance help all studentsregardless of sex, race, color, sexual orientation, gender identity, national origin, or...",36


Remove any comments containing a word from invalid_words. Also remove any comments without any text.

In [5]:
invalid_indexes = []
invalid_words = ['dyslexia', 'disorder', 'attached', 'mental', 'illness', 'ADHD', 'disabled']

# Find rows containing any invalid words
invalid_indexes.extend(data[data['doc.comment_body'].str.contains('|'.join(invalid_words), case=False, na=False)].index.values)

# Check if the comment body is empty
invalid_indexes.extend(data[data['doc.comment_body'] == ''].index.values)

data.drop(invalid_indexes, inplace=True)

Check the most common duplicate comments in the dataset again.

In [6]:
duplicate_data = data.groupby(['doc.comment_body']).size().reset_index(name='duplicate_count')
duplicate_data = duplicate_data.sort_values('duplicate_count', ascending=False).reset_index(drop=True)

with pandas.option_context('display.max_colwidth', 200):
    display(duplicate_data[:5])

,doc.comment_body,duplicate_count
0,"Dear Assistant General Counsel Hilary Malawer,\n\nCurrent federal regulations and guidance help all studentsregardless of sex, race, color, sexual orientation, gender identity, national origin, or...",42
1,"Dear Assistant General Counsel Hilary Malawer,\n\nCurrent federal regulations and guidance help all studentsregardless of sex, race, color, sexual orientation, gender identity, national origin, or...",36
2,"I am commenting on: Regulation ID: ED-2015-OSERS-001-1167 Date Posted: Aug 19, 2016 RIN:1820-AB70\nCFR:34 CFR Parts 361, 363, and 397 Federal Register Number: 2016-15980\nIntegrated AbilityOne (JW...",26
3,"For nearly 50 years, the civil rights of students, including those with disabilities from infancy into adulthood have been the focus of key federal laws passed by the U.S. Congress. These laws, ...",16
4,"Current federal regulations and guidance help all studentsregardless of sex, race, color, sexual orientation, gender identity, national origin, or disability statusreceive the benefit of our civil...",15


See how many comments are left.

In [7]:
len(data)

12225

Export the cleaned up database to be used in other notebooks

In [8]:
data.to_json('./data/comments_cleaned.json', orient='records')